In [ ]:
#Request test block
import requests
from bs4 import BeautifulSoup

url = "https://twitter.com/search?q=depressed&src=typed_query"

response = requests.get(url)

soup = BeautifulSoup(response.text, "html.parser")

print(soup.find_all('p')[0].get_text())

#This is what we get if perform a GET request on Twitter URL : https://twitter.com/search?q=depressed&src=typed_query

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote
import pandas as pd
import datetime

In [ ]:
#This is experiment block
date1 = datetime.datetime.date(datetime.datetime.strptime("2020-01-01", '%Y-%m-%d'))
date2 = datetime.datetime.date(datetime.datetime.strptime("2020-09-19", '%Y-%m-%d'))

days = abs(date1-date2).days
print(days//7)
print(date1 < date2)
print("hey",str(date2 - datetime.timedelta(days=7)))

txt = "https://mobile.twitter.com/search?q=%23twitter+since%3A2020-01-01+until%3A2020-09-19&src=typed_query"
part = txt.split('%3A')
part[2] = 'newwwwwwwwwwwwww&src=typed_query'
txt = '%3A'.join(part)
print(txt)

print(datetime.datetime.date(datetime.datetime.strptime("Sep 11", "%b %d")))

# Search Words/Hashtags, Input Dates

In [ ]:
searchWord = "#twitter"
searchFrom = "2019-01-01"
searchUntil = "2019-12-31"

In [ ]:
class PostsFromTwitter:

    def __init__(self, query, fromDate, toDate):
        
        ''' URL examples
        https://mobile.twitter.com/search?q=twitter%2Canxiety&src=typed_query
        https://mobile.twitter.com/search?q=%23twitter%23anxiety&src=typed_query
        https://mobile.twitter.com/search?q=%23twitter+since%3A2020-01-01+until%3A2020-09-19&src=typed_query
        '''
        
        self.query = quote(query.strip())
        self.fromDate = fromDate
        self.toDate = toDate
        self.numWeeks = self.getNumWeeks()
        
        self.after_part = (
            f"&src=typed_query"
        )
        self.url = f"https://mobile.twitter.com/search?q={self.query}+since%3A{self.fromDate}+until%3A{self.toDate}"
        self.url += self.after_part
        self.tweets = []
    
    def getNumWeeks(self):        #This function returns the number of the weeks between 2 dates.        
        
        self.dateFormat1 = datetime.datetime.date(datetime.datetime.strptime(self.fromDate, '%Y-%m-%d'))
        self.dateFormat2 = datetime.datetime.date(datetime.datetime.strptime(self.toDate, '%Y-%m-%d'))

        return (abs(self.dateFormat1-self.dateFormat2).days)//7

    def scrape_tweets(self):
        
        while self.dateFormat2 > self.dateFormat1 :
            
            self.url = f"https://mobile.twitter.com/search?q=%23depressed+since%3A{self.fromDate}+until%3A{str(self.dateFormat2)}&src=typed_query"
            
            print(f"No. of weeks to traverse : {self.numWeeks} and new URL  : {self.url}")
            
            reloadCount = 40
            while reloadCount > 0 :
                #print("URL to be scraped :: ", self.url)
                content = requests.get(self.url)
                soup = BeautifulSoup(content.text, "html.parser")
                tweet_divs = soup.select("#main_content")[0].select(".tweet")
                #print(tweet_divs)

                for tweet in tweet_divs:
                    handle = tweet.find("div", {"class": "username"}).text.replace("\n", " ").strip()
                    post = tweet.find("div", {"class": "tweet-text"}).text.replace("\n", " ").strip()
                    tweetId = tweet.find("div", {"class": "tweet-text"})['data-id']
                    timestamp = tweet.find("td", {"class": "timestamp"}).find('a').text.replace("\n", " ").strip()
                    self.tweets.append([tweetId,handle,post,timestamp])


                loadMoreButton = soup.select("#main_content")[0].select(".w-button-more")
                try:
                    nextCursorUrl = "https://mobile.twitter.com"+loadMoreButton[0].find('a')['href']
                except IndexError:
                    print("Oops!  list index out of range " , reloadCount)
                    break
                self.url = nextCursorUrl
                reloadCount = reloadCount - 1
            
            self.dateFormat2 = (self.dateFormat2 - datetime.timedelta(days=7)) #1 week less
            self.numWeeks -= 1
            
            
        return self.tweets


### Create an object of PostsFromTwitter usign parameters defined above and call the method scrape_tweets()

In [ ]:
scraperObject = PostsFromTwitter(searchWord,searchFrom,searchUntil)
tweetsList = scraperObject.scrape_tweets()

In [ ]:
print(tweetsList[len(tweetsList)-1])
len(tweetsList)

In [ ]:
df = pd.DataFrame(tweetsList,columns=['tweet_id','@','tweet','date'])
df

In [ ]:
df.to_csv(r'tweetsList.csv',index=False)